In [2]:
import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from IPython import display
import random
import scipy.stats
from sklearn import metrics
from sklearn.metrics import roc_curve
import os

## CIFAR-100, 20k Examples

In [18]:
num_experiments=33
epsilons=[0.5, 1.0, 2.0, 8.0]
epochs= [9] # [7,9]
epochs_nondp = [9] # [5,7,9,30,50]
model="beit_base_patch16_224.in22k_ft_in22k_in1k"
clipping_modes_dp=['BK-MixOpt']
clipping_mode_nondp='nonDP'
num_samples=20000
data='CIFAR100'
exp_types=['ixg', 'sl']
attack_type = 'L1 Norm'

accuracies_df = pd.DataFrame(columns=['epochs', 'epsilon', 'clipping_mode',
                                      'test_acc', 'test_sd',
                                      'train_acc', 'train_sd'
                                     ])
tpr_df = pd.DataFrame(columns=[
    'epochs', 'epsilon', 'clipping_mode', 'explanation_type',
    'tpr_01_mean', 'tpr01_sd',
    'tpr_001_mean', 'tpr_001_sd',
    'auc_mean', 'auc_sd'])

# nonDP
for ep in epochs_nondp:
    acc_file = f'lira/accuracies_{data}_{num_samples}/model={model}_mode={clipping_mode_nondp}_epochs={ep}.txt'
    acc_list = np.loadtxt(acc_file)[:17]
    
    train_acc_file = f'lira/accuracies_train_{data}_{num_samples}/model={model}_mode={clipping_mode_nondp}_epochs={ep}.txt'
    train_acc_list = np.loadtxt(train_acc_file)[:17]
    row = {
        'epochs': ep,
        'epsilon': 0.0,
        'clipping_mode': clipping_mode_nondp,
        'test_acc': np.mean(acc_list),
        'test_sd': np.std(acc_list),
        'train_acc': np.mean(train_acc_list), 
        'train_sd': np.std(train_acc_list)
    }
    accuracies_df.loc[len(accuracies_df)]= list(row.values())
    for exp_type in exp_types:
        tpr01_file = f'lira/tpr01_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        tpr01_list = np.loadtxt(tpr01_file)[:17]
        tpr001_file = f'lira/tpr001_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        tpr001_list = np.loadtxt(tpr001_file)[:17]
        auc_file = f'lira/auc_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        auc_list = np.loadtxt(auc_file)[:17]
        
        row_tpr = {
            'epochs': ep,
            'epsilon': 0.0,
            'clipping_mode': clipping_mode_nondp,
            'explanation_type': exp_type,
            'tpr01_mean': np.mean(tpr01_list),
            'tpr01_sd': np.std(tpr01_list),
            'tpr001_mean': np.mean(tpr001_list),
            'tpr001_sd': np.std(tpr001_list),
            'auc_mean': np.mean(auc_list),
            'auc_sd': np.std(auc_list),
        }
        tpr_df.loc[len(tpr_df)]= list(row_tpr.values())
    
# dp
for ep in epochs:
    for eps in epsilons:
        for clipping_mode_dp in clipping_modes_dp:
            acc_file = f'lira/accuracies_{data}_{num_samples}/model={model}_mode={clipping_mode_dp}_eps={eps}_epochs={ep}.txt'
            acc_list = np.loadtxt(acc_file)[:17]
            acc_list = acc_list[:min(len(acc_list), 17)]
            
            train_acc_file = f'lira/accuracies_train_{data}_{num_samples}/model={model}_mode={clipping_mode_dp}_eps={eps}_epochs={ep}.txt'
            train_acc_list = np.loadtxt(train_acc_file)[:17]
            train_acc_list = train_acc_list[:min(len(train_acc_list), 17)]
            
            row = {
                'epochs': ep,
                'epsilon': eps,
                'clipping_mode': clipping_mode_dp,
                'test_acc': np.mean(acc_list),
                'test_sd': np.std(acc_list),
                'train_acc': np.mean(train_acc_list),
                'train_sd': np.std(train_acc_list)
            }
            accuracies_df.loc[len(accuracies_df)] = list(row.values())
            for exp_type in exp_types:
                tpr01_file = f'lira/tpr01_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_dp}_eps={eps}_type={exp_type}_epochs={ep}.txt'
                tpr01_list = np.loadtxt(tpr01_file)[:min(len(tpr01_list), 17)]
                tpr001_file = f'lira/tpr001_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_dp}_eps={eps}_type={exp_type}_epochs={ep}.txt'
                tpr001_list = np.loadtxt(tpr001_file)[:min(len(tpr001_list), 17)]
                auc_file = f'lira/auc_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_dp}_eps={eps}_type={exp_type}_epochs={ep}.txt'
                auc_list = np.loadtxt(auc_file)[:min(len(auc_list), 17)]
            
                row_tpr = {
                    'epochs': ep,
                    'epsilon': eps,
                    'clipping_mode': clipping_mode_dp,
                    'explanation_type': exp_type,
                    'tpr01_mean': np.mean(tpr01_list),
                    'tpr01_sd': np.std(tpr01_list),
                    'tpr001_mean': np.mean(tpr001_list),
                    'tpr001_sd': np.std(tpr001_list),
                    'auc_mean': np.mean(auc_list),
                    'auc_sd': np.std(auc_list),
                }

                tpr_df.loc[len(tpr_df)]= list(row_tpr.values())
accuracies_df.to_csv(f'tables/accuracies_{data}_{attack_type}.csv')
tpr_df.to_csv(f'tables/tprs_{data}_{attack_type}.csv')
accuracies_df

,epochs,epsilon,clipping_mode,test_acc,test_sd,train_acc,train_sd
0,9,0.0,nonDP,80.108824,0.590363,98.722353,0.321129
1,9,0.5,BK-MixOpt,71.487647,1.176038,73.754118,1.262249
2,9,1.0,BK-MixOpt,77.346471,0.766627,80.081765,0.638217
3,9,2.0,BK-MixOpt,80.907059,0.519647,83.304706,0.566321
4,9,8.0,BK-MixOpt,84.342941,0.546387,86.889412,0.382553


python3 lira_dpmodel_explanations.py --cifar_data CIFAR10 --total_data_examples 20000 --model vit_small_patch16_224 --clipping_mode nonDP --explanation_type sl --dry_run True --experiment_no 1

In [19]:
tpr_df.sort_values(['explanation_type'], ascending=False)

,epochs,epsilon,clipping_mode,explanation_type,tpr_01_mean,tpr01_sd,tpr_001_mean,tpr_001_sd,auc_mean,auc_sd
1,9,0.0,nonDP,sl,0.130753,0.033485,0.019518,0.007872,0.715312,0.009411
3,9,0.5,BK-MixOpt,sl,0.009689,0.001649,0.001360,0.000947,0.499810,0.004134
5,9,1.0,BK-MixOpt,sl,0.012526,0.002164,0.001330,0.000656,0.502011,0.003198
7,9,2.0,BK-MixOpt,sl,0.011549,0.002250,0.001324,0.000905,0.510043,0.004475
9,9,8.0,BK-MixOpt,sl,0.015351,0.003452,0.001513,0.001144,0.522599,0.004211
0,9,0.0,nonDP,ixg,0.133188,0.035204,0.022476,0.008930,0.717132,0.008376
2,9,0.5,BK-MixOpt,ixg,0.010560,0.001568,0.001266,0.000691,0.499356,0.004274
4,9,1.0,BK-MixOpt,ixg,0.010913,0.001781,0.001295,0.000768,0.501771,0.002814
6,9,2.0,BK-MixOpt,ixg,0.012926,0.002491,0.001154,0.000681,0.509005,0.004827
8,9,8.0,BK-MixOpt,ixg,0.015322,0.003061,0.001607,0.000763,0.521949,0.004099


### cifar100: non-DP models only, 33 runs, comparing lira with simple thresholding at low FPR

In [4]:
num_experiments=33
epochs_nondp = [9]
model="beit_base_patch16_224.in22k_ft_in22k_in1k"
# clipping_modes_dp=['BK-MixOpt']
clipping_mode_nondp='nonDP'
num_samples=20000
data='CIFAR100'
exp_types=['ixg', 'sl', 'gs']
attack_type = 'L2 Norm'

tpr_new_vs_simple_df = pd.DataFrame(columns=[
    'explanation_type', 'epochs',
    'tpr_01_mean', 'tpr01_sd',
    'tpr_01_simple_mean', 'tpr01_simple_sd',
    'tpr_001_mean', 'tpr_001_sd',
    'tpr_001_simple_mean', 'tpr_001_simple_sd',
    'auc_mean', 'auc_sd',
    'auc_simple_mean', 'auc_simple_sd'])
# nonDP

for exp_type in tqdm(exp_types):
    for ep in epochs_nondp:
        tpr01_file = f'lira/tpr01_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        tpr01_list = np.loadtxt(tpr01_file)
        tpr001_file = f'lira/tpr001_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        tpr001_list = np.loadtxt(tpr001_file)
        
        tpr01_simple_file = f'lira/tpr01simple_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        tpr01_simple_list = np.loadtxt(tpr01_simple_file)
        tpr001_simple_file = f'lira/tpr001simple_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        tpr001_simple_list = np.loadtxt(tpr001_simple_file)
        
        auc_file = f'lira/auc_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        auc_list = np.loadtxt(auc_file)
        auc_simple_file = f'lira/aucsimple_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        auc_simple_list = np.loadtxt(auc_simple_file)
        
        row_tpr = {
#             'epsilon': 0.0,
#             'clipping_mode': clipping_mode_nondp,
            'explanation_type': exp_type,
            'epochs': ep,
            'tpr01_mean': np.mean(tpr01_list),
            'tpr01_sd': np.std(tpr01_list),
            'tpr01_simple_mean': np.mean(tpr01_simple_list),
            'tpr01_simple_sd': np.std(tpr01_simple_list),
            'tpr001_mean': np.mean(tpr001_list),
            'tpr001_sd': np.std(tpr001_list),
            'tpr001_simple_mean': np.mean(tpr001_simple_list),
            'tpr001_simple_sd': np.std(tpr001_simple_list),
            'auc_mean': np.mean(auc_list),
            'auc_sd': np.std(auc_list),
            'auc_simple_mean': np.mean(auc_simple_list),
            'auc_simple_sd': np.std(auc_simple_list)
        }
        print(row_tpr)
        tpr_new_vs_simple_df.loc[len(tpr_new_vs_simple_df)]= list(row_tpr.values())
    
tpr_new_vs_simple_df.to_csv(f'tables/tprs_nondp_liravssimple_{data}_{attack_type}.csv')
tpr_new_vs_simple_df

  0%|          | 0/3 [00:00<?, ?it/s]

{'explanation_type': 'ixg', 'epochs': 9, 'tpr01_mean': 0.12855454545454545, 'tpr01_sd': 0.03501380758733739, 'tpr01_simple_mean': 0.015712121212121212, 'tpr01_simple_sd': 0.00282647200327681, 'tpr001_mean': 0.021236363636363637, 'tpr001_sd': 0.012385911306725544, 'tpr001_simple_mean': 0.002063636363636364, 'tpr001_simple_sd': 0.0010176676190781105, 'auc_mean': 0.7142859124242426, 'auc_sd': 0.011744302632794896, 'auc_simple_mean': 0.6549935681818182, 'auc_simple_sd': 0.009952501575448625}
{'explanation_type': 'sl', 'epochs': 9, 'tpr01_mean': 0.126903125, 'tpr01_sd': 0.035453239312288165, 'tpr01_simple_mean': 0.015284375000000001, 'tpr01_simple_sd': 0.0030319248604434444, 'tpr001_mean': 0.021875, 'tpr001_sd': 0.010668499191545172, 'tpr001_simple_mean': 0.001696875, 'tpr001_simple_sd': 0.0007152291481581269, 'auc_mean': 0.7138242428125, 'auc_sd': 0.011952689878922724, 'auc_simple_mean': 0.65240529265625, 'auc_simple_sd': 0.009947935099159396}
{'explanation_type': 'gs', 'epochs': 9, 'tpr01

,explanation_type,epochs,tpr_01_mean,tpr01_sd,tpr_01_simple_mean,tpr01_simple_sd,tpr_001_mean,tpr_001_sd,tpr_001_simple_mean,tpr_001_simple_sd,auc_mean,auc_sd,auc_simple_mean,auc_simple_sd
0,ixg,9,0.128555,0.035014,0.015712,0.002826,0.021236,0.012386,0.002064,0.001018,0.714286,0.011744,0.654994,0.009953
1,sl,9,0.126903,0.035453,0.015284,0.003032,0.021875,0.010668,0.001697,0.000715,0.713824,0.011953,0.652405,0.009948
2,gs,9,0.024384,0.004126,0.015274,0.002369,0.003013,0.001297,0.001745,0.001015,0.578864,0.008337,0.585023,0.007417


## CIFAR-10, 20k Examples

In [3]:
num_experiments=33
epsilons=[0.5, 1.0, 2.0, 8.0]
epochs= [30] # [5,10,30]
epochs_nondp = [30] # [5,10,30]
model="vit_small_patch16_224"
# clipping_modes_dp=['BK-MixOpt', 'BK-MixGhostClip']
clipping_modes_dp=['BK-MixOpt']
clipping_mode_nondp='nonDP'
num_samples=20000
data='CIFAR10'
exp_types=['ixg', 'sl']
attack_type = 'L1 Norm'

accuracies_df = pd.DataFrame(columns=['epochs', 'epsilon', 'clipping_mode', 'test_acc', 'test_sd',
                                     'train_acc', 'train_sd'])
tpr_df = pd.DataFrame(columns=[
    'epochs', 'epsilon', 'clipping_mode', 'explanation_type',
    'tpr_01_mean', 'tpr01_sd',
    'tpr_001_mean', 'tpr_001_sd',
    'auc_mean', 'auc_sd'])

# nonDP
for ep in epochs_nondp:
    acc_file = f'lira/accuracies_{data}_{num_samples}/model={model}_mode={clipping_mode_nondp}_epochs={ep}.txt'
    acc_list = np.loadtxt(acc_file)[:17]
    
    train_acc_file = f'lira/accuracies_train_{data}_{num_samples}/model={model}_mode={clipping_mode_nondp}_epochs={ep}.txt'
    train_acc_list = np.loadtxt(train_acc_file)[:17]
    row = {
        'epochs': ep,
        'epsilon': 0.0,
        'clipping_mode': clipping_mode_nondp,
        'test_acc': np.mean(acc_list),
        'test_sd': np.std(acc_list),
        'train_acc': np.mean(train_acc_list),
        'train_sd': np.std(train_acc_list)
    }
    accuracies_df.loc[len(accuracies_df)]= list(row.values())
    for exp_type in exp_types:
        tpr01_file = f'lira/tpr01_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        tpr01_list = np.loadtxt(tpr01_file)
        tpr01_list = tpr01_list[:min(len(tpr01_list), 17)]
#         tpr01_list = np.loadtxt(tpr01_file)
        tpr001_file = f'lira/tpr001_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        tpr001_list = np.loadtxt(tpr001_file)
        tpr001_list = tpr001_list[:min(len(tpr001_list), 17)]
#         tpr001_list = np.loadtxt(tpr001_file)
        auc_file = f'lira/auc_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        auc_list = np.loadtxt(auc_file)
        auc_list = auc_list[:min(len(auc_list), 17)]
#         auc_list = np.loadtxt(auc_file)

        row_tpr = {
            'epochs': ep,
            'epsilon': 0.0,
            'clipping_mode': clipping_mode_nondp,
            'explanation_type': exp_type,
            'tpr01_mean': np.mean(tpr01_list),
            'tpr01_sd': np.std(tpr01_list),
            'tpr001_mean': np.mean(tpr001_list),
            'tpr001_sd': np.std(tpr001_list),
            'auc_mean': np.mean(auc_list),
            'auc_sd': np.std(auc_list),
        }
        tpr_df.loc[len(tpr_df)]= list(row_tpr.values())
    
# dp
for ep in epochs:
    for eps in epsilons:
        for clipping_mode_dp in clipping_modes_dp:
            acc_file = f'lira/accuracies_{data}_{num_samples}/model={model}_mode={clipping_mode_dp}_eps={eps}_epochs={ep}.txt'
            acc_list = np.loadtxt(acc_file)
            acc_list = acc_list[:min(len(acc_list), 17)]
            
            train_acc_file = f'lira/accuracies_train_{data}_{num_samples}/model={model}_mode={clipping_mode_dp}_eps={eps}_epochs={ep}.txt'
            train_acc_list = np.loadtxt(train_acc_file)[:min(len(train_acc_list), 17)]
            row = {
                'epochs': ep,
                'epsilon': eps,
                'clipping_mode': clipping_mode_dp,
                'test_acc': np.mean(acc_list),
                'test_sd': np.std(acc_list),
                'train_acc': np.mean(train_acc_list),
                'train_sd': np.std(train_acc_list)   
            }
            accuracies_df.loc[len(accuracies_df)] = list(row.values())
            for exp_type in exp_types:
                tpr01_file = f'lira/tpr01_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_dp}_eps={eps}_type={exp_type}_epochs={ep}.txt'
                tpr01_list = np.loadtxt(tpr01_file)
                tpr01_list = tpr01_list[:min(len(tpr01_list), 17)]
                tpr001_file = f'lira/tpr001_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_dp}_eps={eps}_type={exp_type}_epochs={ep}.txt'
                tpr001_list = np.loadtxt(tpr001_file)
                tpr001_list = tpr001_list[:min(len(tpr001_list), 17)]
                auc_file = f'lira/auc_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_dp}_eps={eps}_type={exp_type}_epochs={ep}.txt'
                auc_list = np.loadtxt(auc_file)
                auc_list = auc_list[:min(len(auc_list), 17)]

                row_tpr = {
                    'epochs': ep,
                    'epsilon': eps,
                    'clipping_mode': clipping_mode_dp,
                    'explanation_type': exp_type,
                    'tpr01_mean': np.mean(tpr01_list),
                    'tpr01_sd': np.std(tpr01_list),
                    'tpr001_mean': np.mean(tpr001_list),
                    'tpr001_sd': np.std(tpr001_list),
                    'auc_mean': np.mean(auc_list),
                    'auc_sd': np.std(auc_list),
                }

                tpr_df.loc[len(tpr_df)]= list(row_tpr.values())
accuracies_df.to_csv(f'tables/accuracies_{data}_{attack_type}.csv')
tpr_df.to_csv(f'tables/tprs_{data}_{attack_type}.csv')
accuracies_df

,epochs,epsilon,clipping_mode,test_acc,test_sd,train_acc,train_sd
0,30,0.0,nonDP,96.064118,0.612786,100.000000,0.000000
1,30,0.5,BK-MixOpt,95.794118,0.143611,96.311765,0.291915
2,30,1.0,BK-MixOpt,96.231176,0.152812,97.108235,0.198561
3,30,2.0,BK-MixOpt,96.491765,0.158197,97.966471,0.143525
4,30,8.0,BK-MixOpt,96.616471,0.158223,99.028235,0.103027


In [15]:
tpr_df.sort_values('explanation_type', ascending=False)

,epochs,epsilon,clipping_mode,explanation_type,tpr_01_mean,tpr01_sd,tpr_001_mean,tpr_001_sd,auc_mean,auc_sd
1,30,0.0,nonDP,sl,0.152947,0.022982,0.090200,0.022260,0.638569,0.010341
3,30,0.5,BK-MixOpt,sl,0.010242,0.001626,0.001154,0.000554,0.502403,0.004111
5,30,1.0,BK-MixOpt,sl,0.012614,0.002183,0.001642,0.000666,0.509196,0.003144
7,30,2.0,BK-MixOpt,sl,0.016387,0.002948,0.002143,0.001090,0.514182,0.004789
9,30,8.0,BK-MixOpt,sl,0.030485,0.004722,0.004209,0.001634,0.532525,0.004339
0,30,0.0,nonDP,ixg,0.153035,0.021597,0.089476,0.021260,0.638255,0.009525
2,30,0.5,BK-MixOpt,ixg,0.011013,0.002000,0.001389,0.000696,0.502887,0.004261
4,30,1.0,BK-MixOpt,ixg,0.012838,0.002284,0.001884,0.000752,0.508691,0.003498
6,30,2.0,BK-MixOpt,ixg,0.015898,0.002540,0.001889,0.000666,0.514930,0.003967
8,30,8.0,BK-MixOpt,ixg,0.030243,0.004432,0.004044,0.001427,0.532666,0.004360


In [13]:
tpr_df_grouped

,epsilon
explanation_type,
ixg,2.3
sl,2.3


### cifar10: non-DP models only, 33 runs, comparing lira with simple thresholding at low FPR

In [3]:
num_experiments=33
epochs_nondp = [5,10,30]
model="vit_small_patch16_224"
# clipping_modes_dp=['BK-MixOpt']
clipping_mode_nondp='nonDP'
num_samples=20000
data='CIFAR10'
# exp_types = ['ixg']
exp_types=['ixg', 'sl', 'ig', 'gs']
attack_type = 'L2 Norm'

tpr_new_vs_simple_df = pd.DataFrame(columns=[
    'explanation_type', 'epochs',
    'tpr_01_mean', 'tpr01_sd',
    'tpr_01_simple_mean', 'tpr01_simple_sd',
    'tpr_001_mean', 'tpr_001_sd',
    'tpr_001_simple_mean', 'tpr_001_simple_sd',
    'auc_mean', 'auc_sd',
    'auc_simple_mean', 'auc_simple_sd'])
# nonDP

for exp_type in tqdm(exp_types):
    for ep in epochs_nondp:
        tpr01_file = f'lira/tpr01_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        tpr01_list = np.loadtxt(tpr01_file)
        tpr001_file = f'lira/tpr001_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        tpr001_list = np.loadtxt(tpr001_file)
        
        tpr01_simple_file = f'lira/tpr01simple_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        tpr01_simple_list = np.loadtxt(tpr01_simple_file)
        tpr001_simple_file = f'lira/tpr001simple_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        tpr001_simple_list = np.loadtxt(tpr001_simple_file)
        
        auc_file = f'lira/auc_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        auc_list = np.loadtxt(auc_file)
        auc_simple_file = f'lira/aucsimple_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        auc_simple_list = np.loadtxt(auc_simple_file)
        
        row_tpr = {
#             'epsilon': 0.0,
#             'clipping_mode': clipping_mode_nondp,
            'explanation_type': exp_type,
            'epochs': ep,
            'tpr01_mean': np.mean(tpr01_list),
            'tpr01_sd': np.std(tpr01_list),
            'tpr01_simple_mean': np.mean(tpr01_simple_list),
            'tpr01_simple_sd': np.std(tpr01_simple_list),
            'tpr001_mean': np.mean(tpr001_list),
            'tpr001_sd': np.std(tpr001_list),
            'tpr001_simple_mean': np.mean(tpr001_simple_list),
            'tpr001_simple_sd': np.std(tpr001_simple_list),
            'auc_mean': np.mean(auc_list),
            'auc_sd': np.std(auc_list),
            'auc_simple_mean': np.mean(auc_simple_list),
            'auc_simple_sd': np.std(auc_simple_list)
        }
        print(row_tpr)
        tpr_new_vs_simple_df.loc[len(tpr_new_vs_simple_df)]= list(row_tpr.values())
    
tpr_new_vs_simple_df.to_csv(f'tables/tprs_nondp_liravssimple_{data}_{attack_type}.csv')
tpr_new_vs_simple_df

  0%|          | 0/4 [00:00<?, ?it/s]

{'explanation_type': 'ixg', 'epochs': 5, 'tpr01_mean': 0.04161212121212121, 'tpr01_sd': 0.01563269219288312, 'tpr01_simple_mean': 0.011245454545454545, 'tpr01_simple_sd': 0.0020294932804043996, 'tpr001_mean': 0.006466666666666667, 'tpr001_sd': 0.0038871713378883896, 'tpr001_simple_mean': 0.0013030303030303028, 'tpr001_simple_sd': 0.0006483007385207693, 'auc_mean': 0.5639852263636362, 'auc_sd': 0.021007126965281404, 'auc_simple_mean': 0.5379266506060606, 'auc_simple_sd': 0.008982816410141314}
{'explanation_type': 'ixg', 'epochs': 10, 'tpr01_mean': 0.11097878787878789, 'tpr01_sd': 0.01424892888214191, 'tpr01_simple_mean': 0.011606060606060606, 'tpr01_simple_sd': 0.002279279444080013, 'tpr001_mean': 0.03564545454545455, 'tpr001_sd': 0.011345566238766442, 'tpr001_simple_mean': 0.0012515151515151517, 'tpr001_simple_sd': 0.0007105785999733798, 'auc_mean': 0.6151958675757576, 'auc_sd': 0.008476864064130063, 'auc_simple_mean': 0.553518927878788, 'auc_simple_sd': 0.004576528440965882}
{'explana

,explanation_type,epochs,tpr_01_mean,tpr01_sd,tpr_01_simple_mean,tpr01_simple_sd,tpr_001_mean,tpr_001_sd,tpr_001_simple_mean,tpr_001_simple_sd,auc_mean,auc_sd,auc_simple_mean,auc_simple_sd
0,ixg,5,0.041612,0.015633,0.011245,0.002029,0.006467,0.003887,0.001303,0.000648,0.563985,0.021007,0.537927,0.008983
1,ixg,10,0.110979,0.014249,0.011606,0.002279,0.035645,0.011346,0.001252,0.000711,0.615196,0.008477,0.553519,0.004577
2,ixg,30,0.143833,0.016732,0.011136,0.001983,0.073152,0.023631,0.001206,0.000600,0.633850,0.008758,0.558839,0.005476
3,sl,5,0.044342,0.015364,0.010612,0.002087,0.007367,0.003923,0.001191,0.000702,0.564231,0.020930,0.537682,0.009188
4,sl,10,0.111567,0.014029,0.011164,0.002366,0.038824,0.013816,0.001200,0.000587,0.615730,0.008816,0.554026,0.004253
5,sl,30,0.141564,0.018712,0.011070,0.002040,0.073624,0.023607,0.001279,0.000576,0.633107,0.009166,0.559405,0.005015
6,ig,5,0.027088,0.010427,0.010770,0.002405,0.004758,0.002774,0.001206,0.000663,0.532009,0.014232,0.528075,0.007854
7,ig,10,0.058894,0.009119,0.010964,0.002274,0.015636,0.004856,0.001215,0.000727,0.571586,0.008944,0.540536,0.004686
8,ig,30,0.071455,0.012809,0.011009,0.002312,0.019564,0.006206,0.001179,0.000695,0.584981,0.009479,0.544485,0.004592
9,gs,5,0.013512,0.003638,0.011352,0.002768,0.001570,0.000851,0.001300,0.000700,0.515732,0.007850,0.524929,0.007203


## SVHN

In [10]:
epsilons=[0.5, 1.0, 2.0, 8.0]
epochs= [50] # [30,50]
epochs_nondp = [50] # [9,15,30,50]
model="vit_small_patch16_224"
# clipping_modes_dp=['BK-MixOpt', 'BK-MixGhostClip']
clipping_modes_dp=['BK-MixOpt']
clipping_mode_nondp='nonDP'
num_samples=20000
data='SVHN'
exp_types=['ixg', 'ig', 'sl', 'gs']
attack_type = 'Variance'

accuracies_df = pd.DataFrame(columns=['epochs', 'epsilon', 'clipping_mode', 'test_acc', 'test_sd',
                                     'train_acc', 'train_sd'])
tpr_df = pd.DataFrame(columns=[
    'epochs', 'epsilon', 'clipping_mode', 'explanation_type', 'tpr_01_mean', 'tpr01_sd', 'tpr_001_mean', 'tpr_001_sd'])

# nonDP
for ep in epochs_nondp:
    acc_file = f'lira/accuracies_{data}/model={model}_mode={clipping_mode_nondp}_epochs={ep}.txt'
    acc_list = np.loadtxt(acc_file)[:17]
    
    train_acc_file = f'lira/accuracies_train_{data}/model={model}_mode={clipping_mode_nondp}_epochs={ep}.txt'
    train_acc_list = np.loadtxt(train_acc_file)[:17]
    row = {
        'epochs': ep,
        'epsilon': 0.0,
        'clipping_mode': clipping_mode_nondp,
        'test_acc': np.mean(acc_list),
        'test_sd': np.std(acc_list),
        'train_acc': np.mean(train_acc_list),
        'train_sd': np.std(train_acc_list)
    }
    accuracies_df.loc[len(accuracies_df)]= list(row.values())
    for exp_type in exp_types:
        tpr01_file = f'lira/tpr01_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        tpr01_list = np.loadtxt(tpr01_file)
        tpr001_file = f'lira/tpr001_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        tpr001_list = np.loadtxt(tpr001_file)

        row_tpr = {
            'epochs': ep,
            'epsilon': 0.0,
            'clipping_mode': clipping_mode_nondp,
            'explanation_type': exp_type,
            'tpr01_mean': np.mean(tpr01_list),
            'tpr01_sd': np.std(tpr01_list),
            'tpr001_mean': np.mean(tpr001_list),
            'tpr001_sd': np.std(tpr001_list)
        }
        tpr_df.loc[len(tpr_df)]= list(row_tpr.values())
    
# dp
for ep in epochs:
    for eps in epsilons:
        for clipping_mode_dp in clipping_modes_dp:
            acc_file = f'lira/accuracies_{data}/model={model}_mode={clipping_mode_dp}_eps={eps}_epochs={ep}.txt'
            acc_list = np.loadtxt(acc_file)[:17]
            
            train_acc_file = f'lira/accuracies_train_{data}/model={model}_mode={clipping_mode_dp}_eps={eps}_epochs={ep}.txt'
            train_acc_list = np.loadtxt(train_acc_file)[:17]
            row = {
                'epochs': ep,
                'epsilon': eps,
                'clipping_mode': clipping_mode_dp,
                'test_acc': np.mean(acc_list),
                'test_sd': np.std(acc_list),
                'train_acc': np.mean(train_acc_list),
                'train_sd': np.std(train_acc_list)   
            }
            accuracies_df.loc[len(accuracies_df)] = list(row.values())
            for exp_type in exp_types:
                tpr01_file = f'lira/tpr01_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_dp}_eps={eps}_type={exp_type}_epochs={ep}.txt'
                tpr01_list = np.loadtxt(tpr01_file)
                tpr001_file = f'lira/tpr001_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_dp}_eps={eps}_type={exp_type}_epochs={ep}.txt'
                tpr001_list = np.loadtxt(tpr001_file)

                row_tpr = {
                    'epochs': ep,
                    'epsilon': eps,
                    'clipping_mode': clipping_mode_dp,
                    'explanation_type': exp_type,
                    'tpr01_mean': np.mean(tpr01_list),
                    'tpr01_sd': np.std(tpr01_list),
                    'tpr001_mean': np.mean(tpr001_list),
                    'tpr001_sd': np.std(tpr001_list)
                }

                tpr_df.loc[len(tpr_df)]= list(row_tpr.values())
accuracies_df.to_csv(f'tables/accuracies_{data}_{attack_type}.csv')
tpr_df.to_csv(f'tables/tprs_{data}_{attack_type}.csv')
accuracies_df

,epochs,epsilon,clipping_mode,test_acc,test_sd,train_acc,train_sd
0,50,0.0,nonDP,91.557647,1.056217,99.541765,0.223165
1,50,0.5,BK-MixOpt,75.167059,2.399374,76.708235,2.066139
2,50,1.0,BK-MixOpt,84.148824,0.989283,86.152941,0.741388
3,50,2.0,BK-MixOpt,86.922353,0.578340,89.693529,0.378012
4,50,8.0,BK-MixOpt,88.922353,0.405891,93.168235,0.362349


In [12]:
tpr_df

,epochs,epsilon,clipping_mode,explanation_type,tpr_01_mean,tpr01_sd,tpr_001_mean,tpr_001_sd
0,9,0.0,nonDP,ixg,0.011888,0.002360,0.001382,0.000728
1,9,0.0,nonDP,ig,0.011536,0.002333,0.001336,0.000857
2,15,0.0,nonDP,ixg,0.019403,0.004150,0.002836,0.001458
3,15,0.0,nonDP,ig,0.014688,0.002906,0.002091,0.001072
4,30,0.0,nonDP,ixg,0.039624,0.007580,0.006509,0.002708
5,30,0.0,nonDP,ig,0.020873,0.004032,0.003085,0.001416
6,50,0.0,nonDP,ixg,0.056964,0.024099,0.012136,0.010270
7,50,0.0,nonDP,ig,0.025955,0.007081,0.004618,0.002653
8,30,0.5,BK-MixOpt,ixg,0.009500,0.002033,0.000877,0.000377
9,30,0.5,BK-MixOpt,ig,0.010589,0.002062,0.000906,0.000475


### svhn: non-DP models only, 33 runs, comparing lira with simple thresholding at low FPR

In [4]:
num_experiments=33
epochs_nondp = [50]
model="vit_small_patch16_224"
# clipping_modes_dp=['BK-MixOpt']
clipping_mode_nondp='nonDP'
num_samples=20000
data='SVHN'
# exp_types = ['ixg']
exp_types=['ixg', 'sl', 'ig', 'gs']
attack_type = 'L2 Norm'

tpr_new_vs_simple_df = pd.DataFrame(columns=[
    'explanation_type', 'epochs',
    'tpr_01_mean', 'tpr01_sd',
    'tpr_01_simple_mean', 'tpr01_simple_sd',
    'tpr_001_mean', 'tpr_001_sd',
    'tpr_001_simple_mean', 'tpr_001_simple_sd',
    'auc_mean', 'auc_sd',
    'auc_simple_mean', 'auc_simple_sd'])
# nonDP

for exp_type in exp_types:
    for ep in epochs_nondp:
        tpr01_file = f'lira/tpr01_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        tpr01_list = np.loadtxt(tpr01_file)
        tpr001_file = f'lira/tpr001_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        tpr001_list = np.loadtxt(tpr001_file)
        
        tpr01_simple_file = f'lira/tpr01simple_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        tpr01_simple_list = np.loadtxt(tpr01_simple_file)
        tpr001_simple_file = f'lira/tpr001simple_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        tpr001_simple_list = np.loadtxt(tpr001_simple_file)
        
        auc_file = f'lira/auc_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        auc_list = np.loadtxt(auc_file)
        auc_simple_file = f'lira/aucsimple_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        auc_simple_list = np.loadtxt(auc_simple_file)
        
        row_tpr = {
#             'epsilon': 0.0,
#             'clipping_mode': clipping_mode_nondp,
            'explanation_type': exp_type,
            'epochs': ep,
            'tpr01_mean': np.mean(tpr01_list),
            'tpr01_sd': np.std(tpr01_list),
            'tpr01_simple_mean': np.mean(tpr01_simple_list),
            'tpr01_simple_sd': np.std(tpr01_simple_list),
            'tpr001_mean': np.mean(tpr001_list),
            'tpr001_sd': np.std(tpr001_list),
            'tpr001_simple_mean': np.mean(tpr001_simple_list),
            'tpr001_simple_sd': np.std(tpr001_simple_list),
            'auc_mean': np.mean(auc_list),
            'auc_sd': np.std(auc_list),
            'auc_simple_mean': np.mean(auc_simple_list),
            'auc_simple_sd': np.std(auc_simple_list)
        }
        print(row_tpr)
        tpr_new_vs_simple_df.loc[len(tpr_new_vs_simple_df)]= list(row_tpr.values())
    
tpr_new_vs_simple_df.to_csv(f'tables/tprs_nondp_liravssimple_{data}_{attack_type}.csv')
tpr_new_vs_simple_df

{'explanation_type': 'ixg', 'epochs': 50, 'tpr01_mean': 0.06367878787878788, 'tpr01_sd': 0.027705723701132397, 'tpr01_simple_mean': 0.009890909090909089, 'tpr01_simple_sd': 0.001822479768580999, 'tpr001_mean': 0.013615151515151514, 'tpr001_sd': 0.012434607649672169, 'tpr001_simple_mean': 0.001203030303030303, 'tpr001_simple_sd': 0.0009073721125298474, 'auc_mean': 0.598076126060606, 'auc_sd': 0.017817911613714408, 'auc_simple_mean': 0.5447907522727272, 'auc_simple_sd': 0.008038673484548703}
{'explanation_type': 'sl', 'epochs': 50, 'tpr01_mean': 0.07281515151515153, 'tpr01_sd': 0.029070375644778972, 'tpr01_simple_mean': 0.010296969696969697, 'tpr01_simple_sd': 0.00231012066436587, 'tpr001_mean': 0.015706060606060605, 'tpr001_sd': 0.012882733085564672, 'tpr001_simple_mean': 0.0012484848484848486, 'tpr001_simple_sd': 0.0007182137826923259, 'auc_mean': 0.601379539090909, 'auc_sd': 0.01768942205268574, 'auc_simple_mean': 0.5456674868181818, 'auc_simple_sd': 0.007908199711854682}
{'explanatio

,explanation_type,epochs,tpr_01_mean,tpr01_sd,tpr_01_simple_mean,tpr01_simple_sd,tpr_001_mean,tpr_001_sd,tpr_001_simple_mean,tpr_001_simple_sd,auc_mean,auc_sd,auc_simple_mean,auc_simple_sd
0,ixg,50,0.063679,0.027706,0.009891,0.001822,0.013615,0.012435,0.001203,0.000907,0.598076,0.017818,0.544791,0.008039
1,sl,50,0.072815,0.029070,0.010297,0.002310,0.015706,0.012883,0.001248,0.000718,0.601380,0.017689,0.545667,0.007908
2,ig,50,0.028139,0.006906,0.010882,0.002477,0.004782,0.002347,0.001276,0.000664,0.546256,0.008560,0.523251,0.005093
3,gs,50,0.015270,0.003821,0.010318,0.002589,0.002494,0.001328,0.001348,0.000836,0.525616,0.006003,0.522884,0.005380


## GTSRB

In [3]:
epsilons=[0.5, 1.0, 2.0, 8.0]
epochs=[30,50]
epochs_nondp = [9,15,30,50]
model="vit_small_patch16_224"
# clipping_modes_dp=['BK-MixOpt', 'BK-MixGhostClip']
clipping_modes_dp=['BK-MixOpt']
clipping_mode_nondp='nonDP'
num_samples=20000
data='GTSRB'
exp_types=['ixg', 'sl']
attack_type = 'L1 Norm'

accuracies_df = pd.DataFrame(columns=['epochs', 'epsilon', 'clipping_mode', 'test_acc', 'test_sd',
                                     'train_acc', 'train_sd'])
tpr_df = pd.DataFrame(columns=[
    'epochs', 'epsilon', 'clipping_mode', 'explanation_type', 'tpr_01_mean', 'tpr01_sd', 'tpr_001_mean', 'tpr_001_sd'])

# nonDP
for ep in epochs_nondp:
    acc_file = f'lira/accuracies_{data}/model={model}_mode={clipping_mode_nondp}_epochs={ep}.txt'
    acc_list = np.loadtxt(acc_file)[:33]
    
    train_acc_file = f'lira/accuracies_train_{data}/model={model}_mode={clipping_mode_nondp}_epochs={ep}.txt'
    train_acc_list = np.loadtxt(train_acc_file)[:33]
    row = {
        'epochs': ep,
        'epsilon': 0.0,
        'clipping_mode': clipping_mode_nondp,
        'test_acc': np.mean(acc_list),
        'test_sd': np.std(acc_list),
        'train_acc': np.mean(train_acc_list),
        'train_sd': np.std(train_acc_list)
    }
    accuracies_df.loc[len(accuracies_df)]= list(row.values())
    for exp_type in exp_types:
        tpr01_file = f'lira/tpr01_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        tpr01_list = np.loadtxt(tpr01_file)
        tpr001_file = f'lira/tpr001_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        tpr001_list = np.loadtxt(tpr001_file)

        row_tpr = {
            'epochs': ep,
            'epsilon': 0.0,
            'clipping_mode': clipping_mode_nondp,
            'explanation_type': exp_type,
            'tpr01_mean': np.mean(tpr01_list),
            'tpr01_sd': np.std(tpr01_list),
            'tpr001_mean': np.mean(tpr001_list),
            'tpr001_sd': np.std(tpr001_list)
        }
        tpr_df.loc[len(tpr_df)]= list(row_tpr.values())
    
# dp
# for ep in epochs:
#     for eps in epsilons:
#         for clipping_mode_dp in clipping_modes_dp:
#             acc_file = f'lira/accuracies_{data}/model={model}_mode={clipping_mode_dp}_eps={eps}_epochs={ep}.txt'
#             acc_list = np.loadtxt(acc_file)[:16]
            
#             train_acc_file = f'lira/accuracies_train_{data}/model={model}_mode={clipping_mode_dp}_eps={eps}_epochs={ep}.txt'
#             train_acc_list = np.loadtxt(train_acc_file)[:16]
#             row = {
#                 'epochs': ep,
#                 'epsilon': eps,
#                 'clipping_mode': clipping_mode_dp,
#                 'test_acc': np.mean(acc_list),
#                 'test_sd': np.std(acc_list),
#                 'train_acc': np.mean(train_acc_list),
#                 'train_sd': np.std(train_acc_list)   
#             }
#             accuracies_df.loc[len(accuracies_df)] = list(row.values())
#             for exp_type in exp_types:
#                 tpr01_file = f'lira/tpr01_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_dp}_eps={eps}_type={exp_type}_epochs={ep}.txt'
#                 tpr01_list = np.loadtxt(tpr01_file)
#                 tpr001_file = f'lira/tpr001_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_dp}_eps={eps}_type={exp_type}_epochs={ep}.txt'
#                 tpr001_list = np.loadtxt(tpr001_file)

#                 row_tpr = {
#                     'epochs': ep,
#                     'epsilon': eps,
#                     'clipping_mode': clipping_mode_dp,
#                     'explanation_type': exp_type,
#                     'tpr01_mean': np.mean(tpr01_list),
#                     'tpr01_sd': np.std(tpr01_list),
#                     'tpr001_mean': np.mean(tpr001_list),
#                     'tpr001_sd': np.std(tpr001_list)
#                 }

#                 tpr_df.loc[len(tpr_df)]= list(row_tpr.values())
accuracies_df.to_csv(f'tables/accuracies_{data}_{attack_type}.csv')
tpr_df.to_csv(f'tables/tprs_{data}_{attack_type}.csv')
accuracies_df

,epochs,epsilon,clipping_mode,test_acc,test_sd,train_acc,train_sd
0,9,0.0,nonDP,99.546061,0.375023,99.817188,0.334676
1,15,0.0,nonDP,99.623939,0.399036,99.702812,0.625066
2,30,0.0,nonDP,99.813939,0.227262,99.975455,0.050638
3,50,0.0,nonDP,99.903030,0.037778,100.000000,0.000000


### gtsrb: non-DP models only, 33 runs, comparing lira with simple thresholding at low FPR

In [8]:
num_experiments=33
epochs_nondp = [9, 15, 30, 50]
model="vit_small_patch16_224"
# clipping_modes_dp=['BK-MixOpt']
clipping_mode_nondp='nonDP'
num_samples=20000
data='GTSRB'
# exp_types = ['ixg']
exp_types=['ixg', 'sl', 'ig', 'gs']
attack_type = 'Variance'

tpr_new_vs_simple_df = pd.DataFrame(columns=[
    'explanation_type', 'epochs',
    'tpr_01_mean', 'tpr01_sd',
    'tpr_01_simple_mean', 'tpr01_simple_sd',
    'tpr_001_mean', 'tpr_001_sd',
    'tpr_001_simple_mean', 'tpr_001_simple_sd',
    'auc_mean', 'auc_sd',
    'auc_simple_mean', 'auc_simple_sd'])
# nonDP

for exp_type in exp_types:
    for ep in epochs_nondp:
        tpr01_file = f'lira/tpr01_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        tpr01_list = np.loadtxt(tpr01_file)
        tpr001_file = f'lira/tpr001_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        tpr001_list = np.loadtxt(tpr001_file)
        
        tpr01_simple_file = f'lira/tpr01simple_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        tpr01_simple_list = np.loadtxt(tpr01_simple_file)
        tpr001_simple_file = f'lira/tpr001simple_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        tpr001_simple_list = np.loadtxt(tpr001_simple_file)
        
        auc_file = f'lira/auc_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        auc_list = np.loadtxt(auc_file)
        auc_simple_file = f'lira/aucsimple_{data}_{num_samples}_{attack_type}/model={model}_mode={clipping_mode_nondp}_type={exp_type}_epochs={ep}.txt'
        auc_simple_list = np.loadtxt(auc_simple_file)
        
        row_tpr = {
#             'epsilon': 0.0,
#             'clipping_mode': clipping_mode_nondp,
            'explanation_type': exp_type,
            'epochs': ep,
            'tpr01_mean': np.mean(tpr01_list),
            'tpr01_sd': np.std(tpr01_list),
            'tpr01_simple_mean': np.mean(tpr01_simple_list),
            'tpr01_simple_sd': np.std(tpr01_simple_list),
            'tpr001_mean': np.mean(tpr001_list),
            'tpr001_sd': np.std(tpr001_list),
            'tpr001_simple_mean': np.mean(tpr001_simple_list),
            'tpr001_simple_sd': np.std(tpr001_simple_list),
            'auc_mean': np.mean(auc_list),
            'auc_sd': np.std(auc_list),
            'auc_simple_mean': np.mean(auc_simple_list),
            'auc_simple_sd': np.std(auc_simple_list)
        }
        print(row_tpr)
        tpr_new_vs_simple_df.loc[len(tpr_new_vs_simple_df)]= list(row_tpr.values())
    
tpr_new_vs_simple_df.to_csv(f'tables/tprs_nondp_liravssimple_{data}_{attack_type}.csv')
tpr_new_vs_simple_df

{'explanation_type': 'ixg', 'epochs': 9, 'tpr01_mean': 0.012548484848484847, 'tpr01_sd': 0.0027995801862899733, 'tpr01_simple_mean': 0.011136363636363639, 'tpr01_simple_sd': 0.002196114776188357, 'tpr001_mean': 0.0014363636363636362, 'tpr001_sd': 0.0008380417946539284, 'tpr001_simple_mean': 0.0012454545454545455, 'tpr001_simple_sd': 0.0007274242266447014, 'auc_mean': 0.506807096969697, 'auc_sd': 0.005981885582296088, 'auc_simple_mean': 0.5088558693939393, 'auc_simple_sd': 0.004692640361738568}
{'explanation_type': 'ixg', 'epochs': 15, 'tpr01_mean': 0.013575757575757576, 'tpr01_sd': 0.0025455880195885804, 'tpr01_simple_mean': 0.011272727272727271, 'tpr01_simple_sd': 0.0021635090362330956, 'tpr001_mean': 0.0018636363636363635, 'tpr001_sd': 0.0008661367313915077, 'tpr001_simple_mean': 0.0012969696969696968, 'tpr001_simple_sd': 0.0005529451989312506, 'auc_mean': 0.5082749812121212, 'auc_sd': 0.004684415457297547, 'auc_simple_mean': 0.5093770057575757, 'auc_simple_sd': 0.005288295129841574}

,explanation_type,epochs,tpr_01_mean,tpr01_sd,tpr_01_simple_mean,tpr01_simple_sd,tpr_001_mean,tpr_001_sd,tpr_001_simple_mean,tpr_001_simple_sd,auc_mean,auc_sd,auc_simple_mean,auc_simple_sd
0,ixg,9,0.012548,0.002800,0.011136,0.002196,0.001436,0.000838,0.001245,0.000727,0.506807,0.005982,0.508856,0.004693
1,ixg,15,0.013576,0.002546,0.011273,0.002164,0.001864,0.000866,0.001297,0.000553,0.508275,0.004684,0.509377,0.005288
2,ixg,30,0.017145,0.003524,0.010497,0.002006,0.003545,0.001576,0.001333,0.000747,0.512970,0.004742,0.509266,0.005674
3,ixg,50,0.022452,0.004008,0.010670,0.002078,0.006955,0.001809,0.001227,0.000634,0.517292,0.004964,0.510609,0.004773
4,sl,9,0.012855,0.003202,0.010903,0.002085,0.001379,0.000617,0.001133,0.000740,0.507154,0.006138,0.508784,0.004510
5,sl,15,0.014309,0.002983,0.010797,0.002046,0.001742,0.000871,0.001215,0.000703,0.508936,0.005040,0.509503,0.004616
6,sl,30,0.019497,0.004329,0.010267,0.002179,0.004767,0.002084,0.001230,0.000650,0.513394,0.005071,0.509156,0.004759
7,sl,50,0.025824,0.004084,0.010591,0.002097,0.007603,0.002093,0.001400,0.000704,0.517039,0.005094,0.510468,0.004276
8,ig,9,0.012353,0.002615,0.010743,0.002028,0.001730,0.001072,0.001140,0.000685,0.504565,0.004367,0.505431,0.004918
9,ig,15,0.013455,0.002798,0.010952,0.002393,0.001952,0.000927,0.001348,0.000767,0.504794,0.003972,0.505941,0.005110


# 60k Examples

## CIFAR-100, 60k Examples

In [18]:
num_experiments=17
epsilons=[0.5, 1.0, 2.0, 8.0]
epochs=[3,5,7,9]
epochs_nondp = [3,5,7,9]
model="beit_base_patch16_224.in22k_ft_in22k_in1k"
clipping_mode_dp='BK-MixOpt'
clipping_mode_nondp='nonDP'
num_samples=60000
cifar_data='CIFAR100'

accuracies_df = pd.DataFrame(columns=['epochs', 'epsilon', 'clipping_mode', 'acc', 'sd'])
tpr_df = pd.DataFrame(columns=[
    'epochs', 'epsilon', 'clipping_mode', 'tpr_01_mean', 'tpr01_sd', 'tpr_001_mean', 'tpr_001_sd'])

# nonDP
for ep in epochs_nondp:
    acc_file = f'lira/accuracies_{cifar_data}_{num_samples}/model={model}_mode={clipping_mode_nondp}_epochs={ep}.txt'
    acc_list = np.loadtxt(acc_file)
    tpr01_file = f'lira/tpr01_{cifar_data}_{num_samples}/model={model}_mode={clipping_mode_nondp}_epochs={ep}.txt'
    tpr01_list = np.loadtxt(tpr01_file)
    tpr001_file = f'lira/tpr001_{cifar_data}_{num_samples}/model={model}_mode={clipping_mode_nondp}_epochs={ep}.txt'
    tpr001_list = np.loadtxt(tpr001_file)
    row = {
        'epochs': ep,
        'epsilon': 0.0,
        'clipping_mode': clipping_mode_nondp,
        'acc': np.mean(acc_list),
        'sd': np.std(acc_list)
    }
    accuracies_df.loc[len(accuracies_df)]= list(row.values())
    
    row_tpr = {
        'epochs': ep,
        'epsilon': 0.0,
        'clipping_mode': clipping_mode_nondp,
        'tpr01_mean': np.mean(tpr01_list),
        'tpr01_sd': np.std(tpr01_list),
        'tpr001_mean': np.mean(tpr001_list),
        'tpr001_sd': np.std(tpr001_list)
    }
    tpr_df.loc[len(tpr_df)]= list(row_tpr.values())
    
# dp
for ep in epochs:
    for eps in epsilons:
        acc_file = f'lira/accuracies_{cifar_data}_{num_samples}/model={model}_mode={clipping_mode_dp}_eps={eps}_epochs={ep}.txt'
        acc_list = np.loadtxt(acc_file)
        tpr01_file = f'lira/tpr01_{cifar_data}_{num_samples}/model={model}_mode={clipping_mode_dp}_eps={eps}_epochs={ep}.txt'
        tpr01_list = np.loadtxt(tpr01_file)
        tpr001_file = f'lira/tpr001_{cifar_data}_{num_samples}/model={model}_mode={clipping_mode_dp}_eps={eps}_epochs={ep}.txt'
        tpr001_list = np.loadtxt(tpr001_file)
        
        row = {
            'epochs': ep,
            'epsilon': eps,
            'clipping_mode': clipping_mode_dp,
            'acc': np.mean(acc_list),
            'sd': np.std(acc_list)
        }
        accuracies_df.loc[len(accuracies_df)] = list(row.values())
        
        row_tpr = {
            'epochs': ep,
            'epsilon': eps,
            'clipping_mode': clipping_mode_dp,
            'tpr01_mean': np.mean(tpr01_list),
            'tpr01_sd': np.std(tpr01_list),
            'tpr001_mean': np.mean(tpr001_list),
            'tpr001_sd': np.std(tpr001_list)
        }
    
    
        tpr_df.loc[len(tpr_df)]= list(row_tpr.values())
        
accuracies_df

FileNotFoundError: lira/tpr01_CIFAR100_60000/model=beit_base_patch16_224.in22k_ft_in22k_in1k_mode=BK-MixOpt_eps=0.5_epochs=3.txt not found.

In [ ]:
tpr_df

## CIFAR-10, 60k Examples

In [ ]:
num_experiments=33
epsilons=[0.5, 1.0, 2.0, 8.0]
epochs=[5,10,30]
epochs_nondp = [5,10,30]
model="vit_small_patch16_224"
clipping_mode_dp='BK-MixOpt'
clipping_mode_nondp='nonDP'
num_samples=60000
cifar_data='CIFAR10'

accuracies_df = pd.DataFrame(columns=['epochs', 'epsilon', 'clipping_mode', 'acc', 'sd'])
tpr_df = pd.DataFrame(columns=[
    'epochs', 'epsilon', 'clipping_mode', 'tpr_01_mean', 'tpr01_sd', 'tpr_001_mean', 'tpr_001_sd'])

# nonDP
for ep in epochs_nondp:
    acc_file = f'lira/accuracies_{cifar_data}_{num_samples}/model={model}_mode={clipping_mode_nondp}_epochs={ep}.txt'
    acc_list = np.loadtxt(acc_file)
    tpr01_file = f'lira/tpr01_{cifar_data}_{num_samples}/model={model}_mode={clipping_mode_nondp}_epochs={ep}.txt'
    tpr01_list = np.loadtxt(tpr01_file)
    tpr001_file = f'lira/tpr001_{cifar_data}_{num_samples}/model={model}_mode={clipping_mode_nondp}_epochs={ep}.txt'
    tpr001_list = np.loadtxt(tpr001_file)
    row = {
        'epochs': ep,
        'epsilon': 0.0,
        'clipping_mode': clipping_mode_nondp,
        'acc': np.mean(acc_list),
        'sd': np.std(acc_list)
    }
    accuracies_df.loc[len(accuracies_df)]= list(row.values())
    
    row_tpr = {
        'epochs': ep,
        'epsilon': 0.0,
        'clipping_mode': clipping_mode_nondp,
        'tpr01_mean': np.mean(tpr01_list),
        'tpr01_sd': np.std(tpr01_list),
        'tpr001_mean': np.mean(tpr001_list),
        'tpr001_sd': np.std(tpr001_list)
    }
    tpr_df.loc[len(tpr_df)]= list(row_tpr.values())
    
# dp
for ep in epochs:
    for eps in epsilons:
        acc_file = f'lira/accuracies_{cifar_data}_{num_samples}/model={model}_mode={clipping_mode_dp}_eps={eps}_epochs={ep}.txt'
        acc_list = np.loadtxt(acc_file)
        tpr01_file = f'lira/tpr01_{cifar_data}_{num_samples}/model={model}_mode={clipping_mode_dp}_eps={eps}_epochs={ep}.txt'
        tpr01_list = np.loadtxt(tpr01_file)
        tpr001_file = f'lira/tpr001_{cifar_data}_{num_samples}/model={model}_mode={clipping_mode_dp}_eps={eps}_epochs={ep}.txt'
        tpr001_list = np.loadtxt(tpr001_file)
        row = {
            'epochs': ep,
            'epsilon': eps,
            'clipping_mode': clipping_mode_dp,
            'acc': np.mean(acc_list),
            'sd': np.std(acc_list)
        }
        accuracies_df.loc[len(accuracies_df)] = list(row.values())
        
        row_tpr = {
            'epochs': ep,
            'epsilon': eps,
            'clipping_mode': clipping_mode_dp,
            'tpr01_mean': np.mean(tpr01_list),
            'tpr01_sd': np.std(tpr01_list),
            'tpr001_mean': np.mean(tpr001_list),
            'tpr001_sd': np.std(tpr001_list)
        }
    
    
        tpr_df.loc[len(tpr_df)]= list(row_tpr.values())
        
accuracies_df

In [ ]:
tpr_df